# Generate FAQ for README

This notebook shows how to generate multilingual FAQ drafts from a source markdown document.


In [1]:
import json

import pandas as pd
from openai import OpenAI
from pydantic import BaseModel, Field

from openaivec import pandas_ext

pandas_ext.set_client(OpenAI())

pandas_ext.set_responses_model("gpt-5.2")

In [2]:
docs_df: pd.DataFrame = pd.DataFrame(
    {"title": "readme", "body": [open("../../README.md").read()]}
)
docs_df

,title,body
0,readme,# openaivec\n\nAI text processing for pandas a...


In [3]:
class Question(BaseModel):
    question: str = Field(description="The question to ask the model.")
    answer: str = Field(description="The answer to the question.")

class Section(BaseModel):
    title: str = Field(description="The title of the section.")
    content: str = Field(description="The content of the section.")
    questions: list[Question] = Field(description="List of questions and answers related to the section.")

class Document(BaseModel):
    sections: list[Section] = Field(description="List of sections in the document.")

In [4]:
sections_df = docs_df.pipe(
    lambda df: df
    .assign(
        section=lambda df: df["body"].ai.responses(
            instructions="""
            Generate a list of FAQ for each section of the document.
            Break down the document into as many detailed sections as possible,
            regardless of markdown format.
            """,
            response_format=Document
        )
        .map(lambda x: x.sections)
    )
    .drop(columns=["body"])
    .explode("section")
    .ai.extract("section")
)

sections_df


Processing batches:   0%|          | 0/1 [00:00<?, ?item/s]

,title,section_title,section_content,section_questions
0,readme,Project overview: openaivec,AI text processing for pandas and Spark. Apply...,"[{'question': 'What is openaivec?', 'answer': ..."
0,readme,Contributor guidelines link,The README references contributor guidelines i...,[{'question': 'Where are the contributor guide...
0,readme,Quick start: installation,Install from PyPI using `pip install openaivec`.,"[{'question': 'How do I install openaivec?', '..."
0,readme,Quick start: pandas Series example (translate ...,"Shows setting `OPENAI_API_KEY`, creating a pan...",[{'question': 'How do I apply a single prompt ...
0,readme,"Documentation links: tutorials (pandas, spark,...","Provides GitHub Pages tutorials for pandas, Sp...",[{'question': 'Where is the main pandas tutori...
0,readme,Benchmarks: batching and concurrency impact,Benchmark (100 numeric strings → integer liter...,[{'question': 'What does the benchmark measure...
0,readme,Table of contents (navigation),"Lists major sections: Why, Overview, Core Work...",[{'question': 'What are the main topics covere...
0,readme,Why openaivec? (key features),Highlights: drop-in pandas accessors; batch-op...,[{'question': 'What makes openaivec “drop-in” ...
0,readme,Overview (architecture summary),Vectorized batch processing: handle many input...,[{'question': 'How does openaivec ensure outpu...
0,readme,Core workflows: Direct API usage (BatchResponses),Example constructs `BatchResponses.of(client=O...,[{'question': 'When should I use direct API us...


In [5]:
questions_df: pd.DataFrame = sections_df.pipe(
    lambda df: df
    .drop(columns=["section_content"])
    .explode("section_questions")
    .ai.extract("section_questions")
    .reset_index(drop=True)
)


In [6]:
from IPython.display import Markdown, display

display(Markdown(questions_df.to_markdown()))

|    | title   | section_title                                                      | section_questions_question                                                     | section_questions_answer                                                                                                                                                                                                              |
|---:|:--------|:-------------------------------------------------------------------|:-------------------------------------------------------------------------------|:--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
|  0 | readme  | Project overview: openaivec                                        | What is openaivec?                                                             | A Python library that vectorizes LLM text processing for pandas and Spark, letting you apply one prompt or task across many rows efficiently with batching, caching, and retries.                                                     |
|  1 | readme  | Project overview: openaivec                                        | What problem does it solve compared to calling the API row-by-row?             | It reduces HTTP overhead and boilerplate by batching requests, deduplicating repeated inputs, preserving output order, and reusing cached results across workflows.                                                                   |
|  2 | readme  | Project overview: openaivec                                        | Which data tools does it integrate with?                                       | pandas (via `.ai` and `.aio` accessors) and Apache Spark (via UDF helpers and setup functions).                                                                                                                                       |
|  3 | readme  | Project overview: openaivec                                        | Does it support OpenAI and Azure OpenAI?                                       | Yes—both OpenAI API keys and Azure OpenAI (API key or Entra ID) are supported, plus custom client configuration.                                                                                                                      |
|  4 | readme  | Project overview: openaivec                                        | What Python versions are supported?                                            | Python 3.10+.                                                                                                                                                                                                                         |
|  5 | readme  | Contributor guidelines link                                        | Where are the contributor guidelines?                                          | In the `AGENTS.md` file referenced from the README.                                                                                                                                                                                   |
|  6 | readme  | Contributor guidelines link                                        | Do I need to read AGENTS.md before contributing?                               | It’s recommended; it typically contains repo-specific expectations (workflow, coding standards, review process).                                                                                                                      |
|  7 | readme  | Quick start: installation                                          | How do I install openaivec?                                                    | Run `pip install openaivec`.                                                                                                                                                                                                          |
|  8 | readme  | Quick start: installation                                          | Do I need any extras to start?                                                 | For basic pandas usage, the base install is sufficient; for dev work or specific environments (Spark/Fabric), you may need additional dependencies per your setup.                                                                    |
|  9 | readme  | Quick start: pandas Series example (translate fruits)              | How do I apply a single prompt to many pandas values?                          | Use the `.ai.responses(prompt)` accessor on a Series/column to apply one instruction across all rows and return aligned outputs.                                                                                                      |
| 10 | readme  | Quick start: pandas Series example (translate fruits)              | Where do I set my API key for the quick start?                                 | Set `os.environ["OPENAI_API_KEY"] = "your-api-key"` before calling `.ai`/`.aio` methods.                                                                                                                                              |
| 11 | readme  | Quick start: pandas Series example (translate fruits)              | What does `.ai.responses(...)` return?                                         | A vector/Series-like result containing one model response per input row, preserving order.                                                                                                                                            |
| 12 | readme  | Quick start: pandas Series example (translate fruits)              | Do I need to import anything special to enable `.ai` on pandas objects?        | Yes—import `from openaivec import pandas_ext` to register the pandas accessor.                                                                                                                                                        |
| 13 | readme  | Quick start: pandas Series example (translate fruits)              | How do I handle non-string values?                                             | Typically by converting to strings or using tasks/parse utilities depending on your use case; the library is designed for text processing but can be used for structured transformations too.                                         |
| 14 | readme  | Documentation links: tutorials (pandas, spark, prompting, etc.)    | Where is the main pandas tutorial?                                             | https://microsoft.github.io/openaivec/examples/pandas/                                                                                                                                                                                |
| 15 | readme  | Documentation links: tutorials (pandas, spark, prompting, etc.)    | Is there a Spark tutorial?                                                     | Yes: https://microsoft.github.io/openaivec/examples/spark/                                                                                                                                                                            |
| 16 | readme  | Documentation links: tutorials (pandas, spark, prompting, etc.)    | Are there examples for async usage?                                            | Yes: https://microsoft.github.io/openaivec/examples/aio/                                                                                                                                                                              |
| 17 | readme  | Documentation links: tutorials (pandas, spark, prompting, etc.)    | Is there guidance on prompting and few-shot prompts?                           | Yes: https://microsoft.github.io/openaivec/examples/prompt/                                                                                                                                                                           |
| 18 | readme  | Documentation links: tutorials (pandas, spark, prompting, etc.)    | Are there end-to-end example notebooks or pages?                               | Yes—customer analysis, survey transformation, FAQ generation, and a collection page under the examples site.                                                                                                                          |
| 19 | readme  | Benchmarks: batching and concurrency impact                        | What does the benchmark measure?                                               | A simple transformation task (100 numeric strings to integer literals) using `Series.aio.responses` with model `gpt-5.1`, comparing different batching/concurrency settings.                                                          |
| 20 | readme  | Benchmarks: batching and concurrency impact                        | How much faster is batching than serial requests?                              | In the example, serial is ~141s and batching is ~15s—roughly an order-of-magnitude improvement.                                                                                                                                       |
| 21 | readme  | Benchmarks: batching and concurrency impact                        | What is “concurrent batching”?                                                 | It combines request batching with multiple in-flight requests (`max_concurrency`) to overlap network latency and further reduce total runtime.                                                                                        |
| 22 | readme  | Benchmarks: batching and concurrency impact                        | Does batching change the number of outputs returned?                           | No—openaivec still yields one output per input and preserves the original order.                                                                                                                                                      |
| 23 | readme  | Benchmarks: batching and concurrency impact                        | Which settings control performance most directly?                              | `batch_size` and `max_concurrency` (plus model/latency and rate limits).                                                                                                                                                              |
| 24 | readme  | Table of contents (navigation)                                     | What are the main topics covered in the README?                                | Motivation (Why), architecture (Overview), workflows for pandas and direct API usage, authentication (OpenAI/Azure), Spark UDF usage, prompt building tools, Microsoft Fabric notes, contribution workflow, resources, and community. |
| 25 | readme  | Table of contents (navigation)                                     | Where do I find authentication instructions?                                   | In the “pandas authentication options” and “Spark authentication options” sections.                                                                                                                                                   |
| 26 | readme  | Table of contents (navigation)                                     | Where do I learn about prompt tooling?                                         | In the “Building Prompts” section and the linked prompting tutorial.                                                                                                                                                                  |
| 27 | readme  | Why openaivec? (key features)                                      | What makes openaivec “drop-in” for pandas?                                     | It adds `.ai` and `.aio` accessors so analysts can run LLM operations directly on Series/DataFrame columns without changing tools.                                                                                                    |
| 28 | readme  | Why openaivec? (key features)                                      | What do the batching proxies do?                                               | They coalesce multiple inputs into batched API calls, deduplicate repeated inputs, preserve output ordering, and handle failure propagation so waiting callers are unblocked.                                                         |
| 29 | readme  | Why openaivec? (key features)                                      | Does it support structured outputs?                                            | Yes—structured outputs accept Pydantic `response_format` (mirroring OpenAI SDK semantics).                                                                                                                                            |
| 30 | readme  | Why openaivec? (key features)                                      | How does caching help?                                                         | It avoids re-paying for the same prompt+input by reusing results across pandas, Spark, and async flows via shared cache helpers.                                                                                                      |
| 31 | readme  | Why openaivec? (key features)                                      | Is there support for production ETL?                                           | Yes—Spark UDF helpers and Microsoft Fabric guidance are included for scaling beyond notebooks.                                                                                                                                        |
| 32 | readme  | Why openaivec? (key features)                                      | What prompt tooling is included?                                               | `FewShotPromptBuilder` for composing prompts with examples/cautions and `improve()` to iteratively refine prompts.                                                                                                                    |
| 33 | readme  | Why openaivec? (key features)                                      | What are “tasks”?                                                              | Pre-configured, curated prompt+parsing bundles for common operations (e.g., sentiment, intent) that return validated/structured outputs.                                                                                              |
| 34 | readme  | Overview (architecture summary)                                    | How does openaivec ensure outputs match inputs?                                | It enforces ordered outputs and returns one response per input, even when batching/deduplication is used under the hood.                                                                                                              |
| 35 | readme  | Overview (architecture summary)                                    | What happens if the upstream API fails mid-batch?                              | The batching layer is designed to release/unblock waiters on failure so callers don’t hang indefinitely.                                                                                                                              |
| 36 | readme  | Overview (architecture summary)                                    | Can I share caches between pandas and Spark runs?                              | Yes—cache helpers and UDF builders plug into a shared caching layer intended to reuse expensive results across contexts.                                                                                                              |
| 37 | readme  | Overview (architecture summary)                                    | How does it integrate with reasoning models?                                   | It mirrors the OpenAI SDK semantics and allows passing `reasoning` options per request.                                                                                                                                               |
| 38 | readme  | Core workflows: Direct API usage (BatchResponses)                  | When should I use direct API usage instead of pandas/Spark accessors?          | When you want maximum control over batching behavior and request configuration without DataFrame/Series abstractions.                                                                                                                 |
| 39 | readme  | Core workflows: Direct API usage (BatchResponses)                  | What is `BatchResponses`?                                                      | A batching client wrapper that coalesces multiple inputs into efficient OpenAI Responses API calls and returns aligned outputs.                                                                                                       |
| 40 | readme  | Core workflows: Direct API usage (BatchResponses)                  | What does `system_message` do in the example?                                  | It sets a system-level instruction (e.g., strict output format) to guide all responses in that batch client.                                                                                                                          |
| 41 | readme  | Core workflows: Direct API usage (BatchResponses)                  | What does `.parse()` return?                                                   | A list of parsed outputs aligned to the input list (one output per input).                                                                                                                                                            |
| 42 | readme  | Core workflows: Direct API usage (BatchResponses)                  | Can I pass reasoning options here?                                             | Yes—pass a `reasoning` dict (e.g., `{"effort": "none"}`) consistent with the OpenAI SDK argument shape.                                                                                                                               |
| 43 | readme  | pandas authentication options (before using .ai/.aio)              | Do I need to authenticate before calling `.ai` or `.aio`?                      | Yes—set the appropriate environment variables or configure a custom client once before using the accessors.                                                                                                                           |
| 44 | readme  | pandas authentication options (before using .ai/.aio)              | How do I authenticate with OpenAI?                                             | Set `OPENAI_API_KEY` in your environment.                                                                                                                                                                                             |
| 45 | readme  | pandas authentication options (before using .ai/.aio)              | How do I authenticate with Azure OpenAI using an API key?                      | Set `AZURE_OPENAI_API_KEY`, `AZURE_OPENAI_BASE_URL` (ending with `/openai/v1/`), and `AZURE_OPENAI_API_VERSION`.                                                                                                                      |
| 46 | readme  | pandas authentication options (before using .ai/.aio)              | How do I use Azure OpenAI without an API key (Entra ID)?                       | Set `AZURE_OPENAI_BASE_URL` and `AZURE_OPENAI_API_VERSION`, and ensure `AZURE_OPENAI_API_KEY` is unset; openaivec uses `DefaultAzureCredential`.                                                                                      |
| 47 | readme  | pandas authentication options (before using .ai/.aio)              | Can I provide my own OpenAI client instances?                                  | Yes—use `pandas_ext.set_client(OpenAI())` and/or `pandas_ext.set_async_client(AsyncOpenAI())`.                                                                                                                                        |
| 48 | readme  | pandas authentication options (before using .ai/.aio)              | Why would I use a custom client?                                               | To customize networking, retries, proxies, headers, or to share a preconfigured client across your application.                                                                                                                       |
| 49 | readme  | pandas integration (recommended): DataFrame/Series `.ai.responses` | How do I set the default responses model for pandas?                           | Call `pandas_ext.set_responses_model("gpt-5.1")` (or another model/deployment name).                                                                                                                                                  |
| 50 | readme  | pandas integration (recommended): DataFrame/Series `.ai.responses` | How do I add an AI-derived column to a DataFrame?                              | Use `df.assign(newcol=lambda df: df.col.ai.responses(prompt, ...))` to compute a vector of outputs and attach it as a new column.                                                                                                     |
| 51 | readme  | pandas integration (recommended): DataFrame/Series `.ai.responses` | Can I pass per-call options like reasoning?                                    | Yes—pass keyword arguments such as `reasoning={...}` to `.ai.responses`.                                                                                                                                                              |
| 52 | readme  | pandas integration (recommended): DataFrame/Series `.ai.responses` | Does `.ai.responses` preserve row alignment?                                   | Yes—results map 1:1 to input rows in the original order.                                                                                                                                                                              |
| 53 | readme  | Using with reasoning models (pandas)                               | What are “reasoning models” in this context?                                   | Models that expose reasoning-related controls through the OpenAI SDK (e.g., effort), such as o1/o3 variants.                                                                                                                          |
| 54 | readme  | Using with reasoning models (pandas)                               | How do I switch to a reasoning model in pandas?                                | Set it with `pandas_ext.set_responses_model("o1-mini")` (or your chosen reasoning model/deployment).                                                                                                                                  |
| 55 | readme  | Using with reasoning models (pandas)                               | Do I have to pass `reasoning` every time?                                      | No—omit it to use model defaults, or pass it per request to override.                                                                                                                                                                 |
| 56 | readme  | Using with reasoning models (pandas)                               | What shape is the `reasoning` argument?                                        | A dict consistent with the OpenAI SDK, e.g., `{"effort": "none"}`.                                                                                                                                                                    |
| 57 | readme  | Using pre-configured tasks (task library)                          | What is a “task” in openaivec?                                                 | A packaged operation (prompt + expected output schema/validation) for common NLP or business workflows.                                                                                                                               |
| 58 | readme  | Using pre-configured tasks (task library)                          | How do I run a task on a pandas column?                                        | Call `df.col.ai.task(task_object, ...)` where `task_object` comes from `openaivec.task` modules.                                                                                                                                      |
| 59 | readme  | Using pre-configured tasks (task library)                          | What kinds of tasks are included?                                              | Examples shown include sentiment analysis and customer support intent analysis; the library ships a curated set for common text processing.                                                                                           |
| 60 | readme  | Using pre-configured tasks (task library)                          | How do I convert structured task outputs into separate DataFrame columns?      | Use `.ai.extract("field_name")` to extract a named field from structured results into columns.                                                                                                                                        |
| 61 | readme  | Using pre-configured tasks (task library)                          | Can tasks be used with reasoning models?                                       | Yes—pass `reasoning={...}` the same way as with `.responses`.                                                                                                                                                                         |
| 62 | readme  | Asynchronous processing with `.aio`                                | When should I use `.aio` instead of `.ai`?                                     | For high-throughput workloads where async concurrency can reduce end-to-end runtime and better utilize network latency.                                                                                                               |
| 63 | readme  | Asynchronous processing with `.aio`                                | How do I call `.aio.responses`?                                                | Inside an async function, `await series_or_column.aio.responses(prompt, ...)`.                                                                                                                                                        |
| 64 | readme  | Asynchronous processing with `.aio`                                | What does `max_concurrency` control?                                           | The maximum number of concurrent in-flight requests for the async processing call.                                                                                                                                                    |
| 65 | readme  | Asynchronous processing with `.aio`                                | Does async mode still batch requests?                                          | Yes—`.aio` retains automatic batching and deduplication while allowing multiple batches to be processed concurrently.                                                                                                                 |
| 66 | readme  | Asynchronous processing with `.aio`                                | Is async mode safe for large datasets?                                         | It’s designed for high throughput and mentions memory-efficient streaming for large datasets, plus built-in error handling.                                                                                                           |
| 67 | readme  | Using with Apache Spark UDFs (overview)                            | Why use openaivec with Spark?                                                  | To scale LLM-powered transformations across large datasets using distributed Spark execution (ETL/production pipelines).                                                                                                              |
| 68 | readme  | Using with Apache Spark UDFs (overview)                            | How do I learn the Spark integration?                                          | Follow the Spark tutorial: https://microsoft.github.io/openaivec/examples/spark/.                                                                                                                                                     |
| 69 | readme  | Using with Apache Spark UDFs (overview)                            | Does it provide ready-made UDF builders?                                       | Yes—helpers like `responses_udf`, `task_udf`, `embeddings_udf`, and others are provided.                                                                                                                                              |
| 70 | readme  | Spark authentication options and setup                             | How do I configure Spark to use OpenAI?                                        | Call `setup(spark, api_key=..., responses_model_name=..., embeddings_model_name=...)` from `openaivec.spark`.                                                                                                                         |
| 71 | readme  | Spark authentication options and setup                             | How do I configure Spark to use Azure OpenAI deployments?                      | Call `setup_azure(spark, api_key, base_url, api_version, responses_model_name, embeddings_model_name)` using your Azure resource URL and deployment names.                                                                            |
| 72 | readme  | Spark authentication options and setup                             | How do I use Azure OpenAI in Spark with Entra ID?                              | Set `AZURE_OPENAI_BASE_URL` and `AZURE_OPENAI_API_VERSION`, unset `AZURE_OPENAI_API_KEY`, and rely on `DefaultAzureCredential`.                                                                                                       |
| 73 | readme  | Spark authentication options and setup                             | How do I register a UDF that calls the model?                                  | Use `spark.udf.register("udf_name", responses_udf(instructions=..., ...))` and then call it in `selectExpr`/SQL.                                                                                                                      |
| 74 | readme  | Spark authentication options and setup                             | Can I pass reasoning options to Spark UDF calls?                               | Yes—`responses_udf` in the example accepts `reasoning={"effort": "none"}`.                                                                                                                                                            |
| 75 | readme  | Spark authentication options and setup                             | What other Spark UDF helpers exist besides `responses_udf`?                    | `task_udf`, `embeddings_udf`, `count_tokens_udf`, `similarity_udf`, and `parse_udf`.                                                                                                                                                  |
| 76 | readme  | Spark performance tips                                             | Does openaivec deduplicate repeated inputs in Spark?                           | Yes—duplicate detection automatically caches repeated inputs per partition for UDFs.                                                                                                                                                  |
| 77 | readme  | Spark performance tips                                             | What `batch_size` should I use in Spark?                                       | `batch_size=None` for auto-optimization; otherwise try fixed sizes like 32–128 if you need explicit control.                                                                                                                          |
| 78 | readme  | Spark performance tips                                             | How does `max_concurrency` scale in Spark?                                     | It’s per executor, so total concurrency is roughly `num_executors × max_concurrency`.                                                                                                                                                 |
| 79 | readme  | Spark performance tips                                             | What `max_concurrency` should I start with?                                    | The guidance suggests starting around 4–12 and tuning based on throughput and rate limits.                                                                                                                                            |
| 80 | readme  | Spark performance tips                                             | What should I monitor while tuning performance?                                | API rate limits and overall stability; reduce concurrency if you hit throttling or errors.                                                                                                                                            |
| 81 | readme  | Building Prompts (FewShotPromptBuilder and improve)                | What is `FewShotPromptBuilder` used for?                                       | To construct consistent few-shot prompts by specifying a purpose, cautions/constraints, and input/output examples.                                                                                                                    |
| 82 | readme  | Building Prompts (FewShotPromptBuilder and improve)                | What does `.improve()` do?                                                     | It iterates on the prompt (via OpenAI) to refine wording and reduce contradictions; controlled by `max_iter`.                                                                                                                         |
| 83 | readme  | Building Prompts (FewShotPromptBuilder and improve)                | Is `.improve()` required?                                                      | No—it’s optional; you can build prompts directly without refinement.                                                                                                                                                                  |
| 84 | readme  | Building Prompts (FewShotPromptBuilder and improve)                | How do examples help?                                                          | They provide demonstrations of desired behavior/output format, which often improves accuracy and consistency.                                                                                                                         |
| 85 | readme  | Building Prompts (FewShotPromptBuilder and improve)                | Where can I find more prompting guidance?                                      | https://microsoft.github.io/openaivec/examples/prompt/.                                                                                                                                                                               |
| 86 | readme  | Using with Microsoft Fabric                                        | Can I use openaivec in Microsoft Fabric notebooks?                             | Yes—install it from PyPI in your Fabric environment and use the Spark integration as you would in a normal Spark notebook.                                                                                                            |
| 87 | readme  | Using with Microsoft Fabric                                        | Do I need a special API for Fabric?                                            | No special API is mentioned; you use the standard Spark helpers (`openaivec.spark`) within Fabric’s Spark runtime.                                                                                                                    |
| 88 | readme  | Using with Microsoft Fabric                                        | What’s the recommended approach in Fabric for large-scale processing?          | Use Spark-based workflows (UDF helpers/setup) to run LLM transformations at scale.                                                                                                                                                    |
| 89 | readme  | Contributing (development workflow)                                | What is the basic contribution workflow?                                       | Fork the repo, create a branch from `main`, make changes with tests, run lint/format and tests, then open a PR.                                                                                                                       |
| 90 | readme  | Contributing (development workflow)                                | How do I install development dependencies?                                     | Run `uv sync --all-extras --dev`.                                                                                                                                                                                                     |
| 91 | readme  | Contributing (development workflow)                                | How do I lint and auto-fix issues?                                             | Run `uv run ruff check . --fix`.                                                                                                                                                                                                      |
| 92 | readme  | Contributing (development workflow)                                | How do I run a quick test suite?                                               | Run `uv run pytest -m "not slow and not requires_api"`.                                                                                                                                                                               |
| 93 | readme  | Contributing (development workflow)                                | Do I need to add tests when changing code?                                     | Yes—the guidelines explicitly ask to add or update tests when you change code.                                                                                                                                                        |
| 94 | readme  | Additional Resources (example solutions)                           | Where can I find an end-to-end customer feedback analysis example?             | https://microsoft.github.io/openaivec/examples/customer_analysis/.                                                                                                                                                                    |
| 95 | readme  | Additional Resources (example solutions)                           | Is there an example for turning unstructured survey data into structured data? | Yes: https://microsoft.github.io/openaivec/examples/survey_transformation/.                                                                                                                                                           |
| 96 | readme  | Additional Resources (example solutions)                           | Is there an example that demonstrates generating FAQs from documents?          | Yes: https://microsoft.github.io/openaivec/examples/generate_faq/.                                                                                                                                                                    |
| 97 | readme  | Additional Resources (example solutions)                           | Where is the full list of examples?                                            | The examples collection is linked from the README (see “All examples”).                                                                                                                                                               |
| 98 | readme  | Community (Discord)                                                | Is there a community channel for help and announcements?                       | Yes—join the Discord at https://discord.gg/hXCS9J6Qek.                                                                                                                                                                                |
| 99 | readme  | Community (Discord)                                                | What is the Discord used for?                                                  | Support and project announcements.                                                                                                                                                                                                    |

In [7]:
ja_questions_df: pd.DataFrame = questions_df.pipe(
    lambda df: df
    .ai.responses(
        instructions="""
        Translate given json into japanese with same schema.
        Just return the json without any additional text.
        """
    )
    .map(json.loads)
    .ai.extract()
)

Processing batches:   0%|          | 0/100 [00:00<?, ?item/s]

In [8]:
display(Markdown(ja_questions_df.to_markdown()))

|    | record_title   | record_section_title                                                | record_section_questions_question                                            | record_section_questions_answer                                                                                                                                                                                              |
|---:|:---------------|:--------------------------------------------------------------------|:-----------------------------------------------------------------------------|:-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
|  0 | readme         | プロジェクト概要: openaivec                                         | openaivecとは何ですか？                                                      | pandasとSpark向けにLLMのテキスト処理をベクトル化するPythonライブラリです。バッチ処理、キャッシュ、リトライを用いて、多数の行に対して1つのプロンプトやタスクを効率的に適用できます。                                          |
|  1 | readme         | プロジェクト概要: openaivec                                         | APIを行ごとに呼び出すのと比べて、どんな問題を解決しますか？                  | リクエストをバッチ化し、重複入力を排除し、出力順序を保持し、ワークフロー間でキャッシュ結果を再利用することで、HTTPのオーバーヘッドと定型コードを削減します。                                                                 |
|  2 | readme         | プロジェクト概要: openaivec                                         | どのデータツールと統合されていますか？                                       | pandas（`.ai`および`.aio`アクセサ経由）とApache Spark（UDFヘルパーおよびセットアップ関数経由）です。                                                                                                                         |
|  3 | readme         | プロジェクト概要: openaivec                                         | OpenAIとAzure OpenAIの両方に対応していますか？                               | はい。OpenAIのAPIキーとAzure OpenAI（APIキーまたはEntra ID）の両方をサポートしており、カスタムのクライアント設定も可能です。                                                                                                 |
|  4 | readme         | プロジェクト概要: openaivec                                         | 対応しているPythonのバージョンは？                                           | Python 3.10以降です。                                                                                                                                                                                                        |
|  5 | readme         | コントリビューターガイドラインへのリンク                            | コントリビューターガイドラインはどこにありますか？                           | READMEから参照されている`AGENTS.md`ファイルにあります。                                                                                                                                                                      |
|  6 | readme         | コントリビューターガイドラインへのリンク                            | 貢献する前にAGENTS.mdを読む必要がありますか？                                | 推奨されます。通常、リポジトリ固有の期待事項（ワークフロー、コーディング標準、レビュー手順など）が記載されています。                                                                                                         |
|  7 | readme         | クイックスタート: インストール                                      | openaivecはどうやってインストールしますか？                                  | `pip install openaivec`を実行してください。                                                                                                                                                                                  |
|  8 | readme         | クイックスタート: インストール                                      | 開始するために追加要件はありますか？                                         | 基本的なpandas利用であればベースのインストールで十分です。開発作業や特定環境（Spark/Fabric）では、セットアップに応じて追加の依存関係が必要になる場合があります。                                                             |
|  9 | readme         | クイックスタート: pandas Seriesの例（フルーツの翻訳）               | 1つのプロンプトを多くのpandas値に適用するにはどうすればよいですか？          | Series/列で`.ai.responses(prompt)`アクセサを使い、1つの指示を全行に適用して、対応する出力を整列した形で返します。                                                                                                            |
| 10 | readme         | クイックスタート：pandas Series の例（フルーツを翻訳）              | クイックスタートではどこでAPIキーを設定しますか？                            | `.ai`/`.aio` メソッドを呼び出す前に、`os.environ["OPENAI_API_KEY"] = "your-api-key"` を設定してください。                                                                                                                    |
| 11 | readme         | クイックスタート：pandas Series の例（フルーツを翻訳）              | `.ai.responses(...)` は何を返しますか？                                      | 入力行ごとに1つのモデル応答を含み、順序を保持するベクトル/Series風の結果を返します。                                                                                                                                         |
| 12 | readme         | クイックスタート：pandas Series の例（フルーツを翻訳）              | pandasオブジェクトで `.ai` を有効にするために特別なインポートは必要ですか？  | はい。pandasアクセサを登録するために `from openaivec import pandas_ext` をインポートしてください。                                                                                                                           |
| 13 | readme         | クイックスタート：pandas Series の例（フルーツを翻訳）              | 文字列でない値はどう扱えばよいですか？                                       | 通常は文字列に変換するか、用途に応じてタスク/parseユーティリティを使います。このライブラリはテキスト処理向けに設計されていますが、構造化された変換にも使用できます。                                                         |
| 14 | readme         | ドキュメントリンク：チュートリアル（pandas、spark、プロンプトなど） | 主要なpandasチュートリアルはどこにありますか？                               | https://microsoft.github.io/openaivec/examples/pandas/                                                                                                                                                                       |
| 15 | readme         | ドキュメントリンク：チュートリアル（pandas、spark、プロンプトなど） | Sparkのチュートリアルはありますか？                                          | はい：https://microsoft.github.io/openaivec/examples/spark/                                                                                                                                                                  |
| 16 | readme         | ドキュメントリンク：チュートリアル（pandas、spark、プロンプトなど） | 非同期利用の例はありますか？                                                 | はい：https://microsoft.github.io/openaivec/examples/aio/                                                                                                                                                                    |
| 17 | readme         | ドキュメントリンク：チュートリアル（pandas、spark、プロンプトなど） | プロンプト設計やfew-shotプロンプトのガイダンスはありますか？                 | はい：https://microsoft.github.io/openaivec/examples/prompt/                                                                                                                                                                 |
| 18 | readme         | ドキュメントリンク：チュートリアル（pandas、spark、プロンプトなど） | エンドツーエンドの例のノートブックやページはありますか？                     | はい。顧客分析、アンケート変換、FAQ生成、および examples サイト配下のコレクションページがあります。                                                                                                                          |
| 19 | readme         | ベンチマーク：バッチ処理と並行性の影響                              | ベンチマークは何を測定していますか？                                         | `Series.aio.responses` とモデル `gpt-5.1` を用いた簡単な変換タスク（数値文字列100件を整数リテラルに変換）を対象に、異なるバッチ処理/並行性設定を比較します。                                                                 |
| 20 | readme         | ベンチマーク：バッチ処理と並行実行の影響                            | バッチ処理はシリアルリクエストよりどれくらい速いですか？                     | 例では、シリアルが約141秒、バッチ処理が約15秒で、概ね1桁（約10倍）の改善です。                                                                                                                                               |
| 21 | readme         | ベンチマーク：バッチ処理と並行実行の影響                            | 「並行バッチ処理（concurrent batching）」とは何ですか？                      | リクエストのバッチ化に加えて、複数のインフライトリクエスト（`max_concurrency`）を同時に走らせ、ネットワーク待ち時間を重ね合わせて総実行時間をさらに短縮します。                                                              |
| 22 | readme         | ベンチマーク：バッチ処理と並行実行の影響                            | バッチ処理で返される出力数は変わりますか？                                   | いいえ。openaivec は入力1件につき出力1件を返し、元の順序も保持します。                                                                                                                                                       |
| 23 | readme         | ベンチマーク：バッチ処理と並行実行の影響                            | パフォーマンスを最も直接的に制御する設定はどれですか？                       | 主に `batch_size` と `max_concurrency`（加えてモデル/レイテンシやレート制限）です。                                                                                                                                          |
| 24 | readme         | 目次（ナビゲーション）                                              | README で扱っている主なトピックは何ですか？                                  | 動機（Why）、アーキテクチャ（Overview）、pandas と直接API利用のワークフロー、認証（OpenAI/Azure）、Spark UDF の利用、プロンプト構築ツール、Microsoft Fabric の注意点、コントリビューション手順、リソース、コミュニティです。 |
| 25 | readme         | 目次（ナビゲーション）                                              | 認証手順はどこにありますか？                                                 | 「pandas の認証オプション」と「Spark の認証オプション」のセクションにあります。                                                                                                                                              |
| 26 | readme         | 目次（ナビゲーション）                                              | プロンプト用ツールについてはどこで学べますか？                               | 「Building Prompts」セクションと、リンクされているプロンプティングのチュートリアルです。                                                                                                                                     |
| 27 | readme         | なぜ openaivec？（主な機能）                                        | openaivec が pandas に「ドロップイン」だと言えるのはなぜですか？             | `.ai` と `.aio` アクセサを追加し、分析者がツールを変えずに Series/DataFrame の列に対して直接 LLM 操作を実行できるようにするからです。                                                                                        |
| 28 | readme         | なぜ openaivec？（主な機能）                                        | バッチング用プロキシは何をしますか？                                         | 複数の入力をまとめてバッチAPI呼び出しに統合し、重複入力を排除し、出力順序を保持し、失敗の伝播を扱って待機中の呼び出し元がブロック解除されるようにします。                                                                    |
| 29 | readme         | なぜ openaivec？（主な機能）                                        | 構造化出力に対応していますか？                                               | はい。構造化出力は Pydantic の `response_format` を受け付け（OpenAI SDK のセマンティクスを踏襲します）。                                                                                                                     |
| 30 | readme         | なぜopenaivecなのか？（主な特徴）                                   | キャッシュはどのように役立ちますか？                                         | 共有キャッシュヘルパーを通じて、pandas・Spark・非同期フロー間で結果を再利用し、同じプロンプト+入力に対して再度支払うことを避けられます。                                                                                     |
| 31 | readme         | なぜopenaivecなのか？（主な特徴）                                   | 本番ETLのサポートはありますか？                                              | はい。ノートブックを超えてスケールできるよう、Spark UDFヘルパーとMicrosoft Fabric向けのガイダンスが含まれています。                                                                                                          |
| 32 | readme         | なぜopenaivecなのか？（主な特徴）                                   | どのようなプロンプト用ツールが含まれていますか？                             | 例や注意事項を含めてプロンプトを構成するための`FewShotPromptBuilder`と、プロンプトを反復的に改善する`improve()`が含まれています。                                                                                            |
| 33 | readme         | なぜopenaivecなのか？（主な特徴）                                   | 「タスク」とは何ですか？                                                     | よくある操作（例：感情、意図）向けに、プロンプト+パース処理を事前設定・厳選したバンドルで、検証済み／構造化された出力を返します。                                                                                            |
| 34 | readme         | 概要（アーキテクチャ要約）                                          | openaivecは出力が入力と一致することをどのように保証しますか？                | 内部でバッチ処理／重複排除を行っていても、出力順序を強制し、入力ごとに1つの応答を返します。                                                                                                                                  |
| 35 | readme         | 概要（アーキテクチャ要約）                                          | 上流APIがバッチの途中で失敗した場合はどうなりますか？                        | バッチ層は、失敗時に待機者を解放／ブロック解除するよう設計されており、呼び出し側が無期限にハングしないようにします。                                                                                                         |
| 36 | readme         | 概要（アーキテクチャ要約）                                          | pandasとSparkの実行間でキャッシュを共有できますか？                          | はい。キャッシュヘルパーとUDFビルダーは、コンテキストをまたいで高コストな結果を再利用することを意図した共有キャッシュ層に接続します。                                                                                        |
| 37 | readme         | 概要（アーキテクチャ要約）                                          | 推論モデルとはどのように統合しますか？                                       | OpenAI SDKのセマンティクスを踏襲しており、リクエストごとに`reasoning`オプションを渡せます。                                                                                                                                  |
| 38 | readme         | コアワークフロー：直接API利用（BatchResponses）                     | pandas/Sparkアクセサの代わりに直接API利用を使うのはどんなときですか？        | DataFrame/Seriesの抽象化なしに、バッチ動作やリクエスト設定を最大限コントロールしたい場合です。                                                                                                                               |
| 39 | readme         | コアワークフロー：直接API利用（BatchResponses）                     | `BatchResponses`とは何ですか？                                               | 複数入力を効率的なOpenAI Responses API呼び出しに集約し、整列された出力を返すバッチング用クライアントラッパーです。                                                                                                           |
| 40 | readme         | コアワークフロー: 直接API利用（BatchResponses）                     | 例の中で `system_message` は何をしますか？                                   | バッチクライアント内のすべての応答を導くためのシステムレベルの指示（例: 厳密な出力形式）を設定します。                                                                                                                       |
| 41 | readme         | コアワークフロー: 直接API利用（BatchResponses）                     | `.parse()` は何を返しますか？                                                | 入力リストに対応して整列された解析済み出力のリスト（入力1件につき出力1件）を返します。                                                                                                                                       |
| 42 | readme         | コアワークフロー: 直接API利用（BatchResponses）                     | ここで reasoning オプションを渡せますか？                                    | はい。OpenAI SDK の引数形状に一致する `reasoning` 辞書（例: `{"effort": "none"}`）を渡せます。                                                                                                                               |
| 43 | readme         | pandas の認証オプション（.ai/.aio を使う前）                        | `.ai` や `.aio` を呼ぶ前に認証は必要ですか？                                 | はい。アクセサを使う前に、適切な環境変数を設定するか、カスタムクライアントを一度だけ設定してください。                                                                                                                       |
| 44 | readme         | pandas の認証オプション（.ai/.aio を使う前）                        | OpenAI での認証はどうすればよいですか？                                      | 環境に `OPENAI_API_KEY` を設定してください。                                                                                                                                                                                 |
| 45 | readme         | pandas の認証オプション（.ai/.aio を使う前）                        | APIキーで Azure OpenAI に認証するにはどうすればよいですか？                  | `AZURE_OPENAI_API_KEY`、`AZURE_OPENAI_BASE_URL`（末尾が `/openai/v1/`）および `AZURE_OPENAI_API_VERSION` を設定してください。                                                                                                |
| 46 | readme         | pandas の認証オプション（.ai/.aio を使う前）                        | APIキーなし（Entra ID）で Azure OpenAI を使うにはどうすればよいですか？      | `AZURE_OPENAI_BASE_URL` と `AZURE_OPENAI_API_VERSION` を設定し、`AZURE_OPENAI_API_KEY` が未設定であることを確認してください。openaivec は `DefaultAzureCredential` を使用します。                                            |
| 47 | readme         | pandas の認証オプション（.ai/.aio を使う前）                        | 自分の OpenAI クライアントインスタンスを渡せますか？                         | はい。`pandas_ext.set_client(OpenAI())` および/または `pandas_ext.set_async_client(AsyncOpenAI())` を使用してください。                                                                                                      |
| 48 | readme         | pandas の認証オプション（.ai/.aio を使う前）                        | なぜカスタムクライアントを使うのですか？                                     | ネットワーク設定、リトライ、プロキシ、ヘッダーをカスタマイズしたり、事前設定済みクライアントをアプリケーション全体で共有したりするためです。                                                                                 |
| 49 | readme         | pandas 連携（推奨）: DataFrame/Series の `.ai.responses`            | pandas の既定の responses モデルを設定するにはどうすればよいですか？         | `pandas_ext.set_responses_model("gpt-5.1")`（または別のモデル/デプロイ名）を呼び出してください。                                                                                                                             |
| 50 | readme         | pandas 連携（推奨）: DataFrame/Series の `.ai.responses`            | DataFrame に AI 由来の列を追加するにはどうすればよいですか？                 | `df.assign(newcol=lambda df: df.col.ai.responses(prompt, ...))` を使って出力ベクトルを計算し、新しい列として付与してください。                                                                                               |
| 51 | readme         | pandas 連携（推奨）：DataFrame/Series の `.ai.responses`            | 推論（reasoning）などの呼び出しごとのオプションは渡せますか？                | はい。`.ai.responses` に `reasoning={...}` などのキーワード引数を渡してください。                                                                                                                                            |
| 52 | readme         | pandas 連携（推奨）：DataFrame/Series の `.ai.responses`            | `.ai.responses` は行の対応（アラインメント）を維持しますか？                 | はい。結果は元の順序のまま、入力行に対して 1:1 で対応します。                                                                                                                                                                |
| 53 | readme         | 推論モデルの使用（pandas）                                          | この文脈での「推論モデル」とは何ですか？                                     | OpenAI SDK を通じて推論関連の制御（例：effort）を公開するモデル（o1/o3 系のバリアントなど）です。                                                                                                                            |
| 54 | readme         | 推論モデルの使用（pandas）                                          | pandas で推論モデルに切り替えるには？                                        | `pandas_ext.set_responses_model("o1-mini")`（または選択した推論モデル/デプロイメント）で設定します。                                                                                                                         |
| 55 | readme         | 推論モデルの使用（pandas）                                          | 毎回 `reasoning` を渡す必要がありますか？                                    | いいえ。省略すればモデルのデフォルトが使われます。上書きしたい場合はリクエストごとに渡してください。                                                                                                                         |
| 56 | readme         | 推論モデルの使用（pandas）                                          | `reasoning` 引数の形（形式）は？                                             | OpenAI SDK と整合する dict です。例：`{"effort": "none"}`。                                                                                                                                                                  |
| 57 | readme         | 事前設定タスクの使用（タスクライブラリ）                            | openaivec における「タスク」とは何ですか？                                   | 一般的な NLP や業務ワークフロー向けに、（プロンプト＋期待する出力スキーマ/検証）をパッケージ化した処理です。                                                                                                                 |
| 58 | readme         | 事前設定タスクの使用（タスクライブラリ）                            | pandas の列に対してタスクを実行するには？                                    | `openaivec.task` モジュール群から取得した `task_object` を使い、`df.col.ai.task(task_object, ...)` を呼び出します。                                                                                                          |
| 59 | readme         | 事前設定タスクの使用（タスクライブラリ）                            | どんな種類のタスクが含まれていますか？                                       | 例としては感情分析やカスタマーサポートの意図分析が示されています。ライブラリには一般的なテキスト処理向けの厳選セットが同梱されています。                                                                                     |
| 60 | readme         | 事前設定タスクの使用（タスクライブラリ）                            | 構造化されたタスク出力を DataFrame の別列に変換するには？                    | 構造化結果から名前付きフィールドを列として取り出すには、`.ai.extract("field_name")` を使います。                                                                                                                             |
| 61 | readme         | 事前設定タスクの使用（タスクライブラリ）                            | タスクは推論モデルでも使えますか？                                           | はい。`.responses` と同様に `reasoning={...}` を渡してください。                                                                                                                                                             |
| 62 | readme         | `.aio` による非同期処理                                             | `.ai` ではなく `.aio` を使うべきなのはいつですか？                           | 非同期の並行処理によってエンドツーエンドの実行時間を短縮でき、ネットワーク待ち時間をより有効活用できるような高スループットのワークロードの場合です。                                                                         |
| 63 | readme         | `.aio` による非同期処理                                             | `.aio.responses` はどのように呼び出しますか？                                | async 関数の中で、`await series_or_column.aio.responses(prompt, ...)` を呼び出します。                                                                                                                                       |
| 64 | readme         | `.aio` による非同期処理                                             | `max_concurrency` は何を制御しますか？                                       | 非同期処理呼び出しにおける、同時に処理中（in-flight）となるリクエスト数の上限です。                                                                                                                                          |
| 65 | readme         | `.aio` による非同期処理                                             | 非同期モードでもリクエストはバッチ化されますか？                             | はい。`.aio` は自動バッチ化と重複排除を維持しつつ、複数のバッチを並行して処理できるようにします。                                                                                                                            |
| 66 | readme         | `.aio` による非同期処理                                             | 非同期モードは大規模データセットでも安全ですか？                             | 高スループット向けに設計されており、大規模データセット向けのメモリ効率のよいストリーミングに加えて、組み込みのエラーハンドリングも言及されています。                                                                         |
| 67 | readme         | Apache Spark UDF と一緒に使う（概要）                               | Spark で openaivec を使う理由は何ですか？                                    | 分散 Spark 実行（ETL／本番パイプライン）を用いて、大規模データセットに対する LLM 活用の変換処理をスケールさせるためです。                                                                                                    |
| 68 | readme         | Apache Spark UDF と一緒に使う（概要）                               | Spark 連携はどうやって学べますか？                                           | Spark チュートリアルに従ってください: https://microsoft.github.io/openaivec/examples/spark/.                                                                                                                                 |
| 69 | readme         | Apache Spark UDF と一緒に使う（概要）                               | すぐ使える UDF ビルダーは提供されていますか？                                | はい。`responses_udf`、`task_udf`、`embeddings_udf` などのヘルパーが提供されています。                                                                                                                                       |
| 70 | readme         | Spark の認証オプションとセットアップ                                | Spark で OpenAI を使うにはどう設定しますか？                                 | `openaivec.spark` の `setup(spark, api_key=..., responses_model_name=..., embeddings_model_name=...)` を呼び出します。                                                                                                       |
| 71 | readme         | Spark の認証オプションとセットアップ                                | Spark で Azure OpenAI のデプロイメントを使うにはどう設定しますか？           | Azure リソース URL とデプロイ名を用いて、`setup_azure(spark, api_key, base_url, api_version, responses_model_name, embeddings_model_name)` を呼び出します。                                                                  |
| 72 | readme         | Spark の認証オプションとセットアップ                                | Spark で Entra ID を使って Azure OpenAI を利用するにはどうすればよいですか？ | `AZURE_OPENAI_BASE_URL` と `AZURE_OPENAI_API_VERSION` を設定し、`AZURE_OPENAI_API_KEY` を未設定（unset）にして、`DefaultAzureCredential` に依存します。                                                                      |
| 73 | readme         | Sparkの認証オプションとセットアップ                                 | モデルを呼び出すUDFを登録するにはどうすればよいですか？                      | `spark.udf.register("udf_name", responses_udf(instructions=..., ...))` を使用し、その後 `selectExpr`/SQL で呼び出します。                                                                                                    |
| 74 | readme         | Sparkの認証オプションとセットアップ                                 | Spark UDF呼び出しに推論オプションを渡せますか？                              | はい—例の `responses_udf` は `reasoning={"effort": "none"}` を受け取ります。                                                                                                                                                 |
| 75 | readme         | Sparkの認証オプションとセットアップ                                 | `responses_udf` 以外にどんなSpark UDFヘルパーがありますか？                  | `task_udf`、`embeddings_udf`、`count_tokens_udf`、`similarity_udf`、`parse_udf` があります。                                                                                                                                 |
| 76 | readme         | Sparkのパフォーマンスのヒント                                       | openaivecはSparkで繰り返し入力を重複排除しますか？                           | はい—重複検出により、UDFのパーティションごとに繰り返し入力が自動的にキャッシュされます。                                                                                                                                     |
| 77 | readme         | Sparkのパフォーマンスのヒント                                       | Sparkではどの `batch_size` を使うべきですか？                                | 自動最適化には `batch_size=None`。明示的に制御したい場合は、32〜128 のような固定サイズを試してください。                                                                                                                     |
| 78 | readme         | Sparkのパフォーマンスのヒント                                       | Sparkでは `max_concurrency` はどのようにスケールしますか？                   | エグゼキュータ単位なので、総並行数は概ね `num_executors × max_concurrency` です。                                                                                                                                            |
| 79 | readme         | Sparkのパフォーマンスのヒント                                       | `max_concurrency` はいくつから始めるべきですか？                             | ガイダンスでは、4〜12 程度から始め、スループットとレート制限に基づいて調整することが推奨されています。                                                                                                                       |
| 80 | readme         | Sparkのパフォーマンスのヒント                                       | パフォーマンス調整中に何を監視すべきですか？                                 | APIのレート制限と全体の安定性です。スロットリングやエラーが出たら並行数を下げてください。                                                                                                                                    |
| 81 | readme         | プロンプトの構築（FewShotPromptBuilder と improve）                 | `FewShotPromptBuilder` は何に使いますか？                                    | 目的、注意点/制約、入出力の例を指定して、一貫したfew-shotプロンプトを構築するために使います。                                                                                                                                |
| 82 | readme         | プロンプトの構築（FewShotPromptBuilder と improve）                 | `.improve()` は何をしますか？                                                | （OpenAI経由で）プロンプトを反復的に改善し、表現を洗練して矛盾を減らします。`max_iter` で制御します。                                                                                                                        |
| 83 | readme         | プロンプトの構築（FewShotPromptBuilder と improve）                 | `.improve()` は必須ですか？                                                  | いいえ—任意です。改善なしで直接プロンプトを構築できます。                                                                                                                                                                    |
| 84 | readme         | プロンプトの構築（FewShotPromptBuilder と improve）                 | 例はどのように役立ちますか？                                                 | 望ましい振る舞い／出力形式のデモンストレーションを提供するため、精度と一貫性が向上することが多いです。                                                                                                                       |
| 85 | readme         | プロンプトの構築（FewShotPromptBuilder と improve）                 | プロンプティングに関する追加のガイダンスはどこで入手できますか？             | https://microsoft.github.io/openaivec/examples/prompt/.                                                                                                                                                                      |
| 86 | readme         | Microsoft Fabric での使用                                           | Microsoft Fabric のノートブックで openaivec を使えますか？                   | はい。Fabric 環境で PyPI からインストールし、通常の Spark ノートブックと同様に Spark 統合を使用できます。                                                                                                                    |
| 87 | readme         | Microsoft Fabric での使用                                           | Fabric 用に特別な API は必要ですか？                                         | 特別な API は言及されていません。Fabric の Spark ランタイム内で標準の Spark ヘルパー（`openaivec.spark`）を使用します。                                                                                                      |
| 88 | readme         | Microsoft Fabric での使用                                           | Fabric で大規模処理を行う推奨アプローチは何ですか？                          | Spark ベースのワークフロー（UDF ヘルパー／セットアップ）を使用して、LLM 変換をスケールさせて実行します。                                                                                                                     |
| 89 | readme         | コントリビュート（開発ワークフロー）                                | 基本的なコントリビューションのワークフローは何ですか？                       | リポジトリをフォークし、`main` からブランチを作成し、テスト付きで変更を行い、lint/format とテストを実行してから PR を作成します。                                                                                            |
| 90 | readme         | コントリビュート（開発ワークフロー）                                | 開発用依存関係はどのようにインストールしますか？                             | `uv sync --all-extras --dev` を実行します。                                                                                                                                                                                  |
| 91 | readme         | コントリビュート（開発ワークフロー）                                | lint を実行して問題を自動修正するにはどうすればよいですか？                  | `uv run ruff check . --fix` を実行します。                                                                                                                                                                                   |
| 92 | readme         | コントリビュート（開発ワークフロー）                                | 簡易なテストスイートはどのように実行しますか？                               | `uv run pytest -m "not slow and not requires_api"` を実行します。                                                                                                                                                            |
| 93 | readme         | コントリビュート（開発ワークフロー）                                | コードを変更する際にテストを追加する必要はありますか？                       | はい。ガイドラインでは、コードを変更した場合にテストを追加または更新するよう明確に求めています。                                                                                                                             |
| 94 | readme         | 追加リソース（サンプルソリューション）                              | エンドツーエンドの顧客フィードバック分析の例はどこで見つけられますか？       | https://microsoft.github.io/openaivec/examples/customer_analysis/.                                                                                                                                                           |
| 95 | readme         | 追加リソース（サンプルソリューション）                              | 非構造化のアンケートデータを構造化データに変換する例はありますか？           | はい: https://microsoft.github.io/openaivec/examples/survey_transformation/.                                                                                                                                                 |
| 96 | readme         | 追加リソース（サンプルソリューション）                              | ドキュメントからFAQを生成する例はありますか？                                | はい: https://microsoft.github.io/openaivec/examples/generate_faq/.                                                                                                                                                          |
| 97 | readme         | 追加リソース（サンプルソリューション）                              | サンプルの全一覧はどこにありますか？                                         | サンプル集へのリンクはREADMEにあります（「All examples」を参照）。                                                                                                                                                           |
| 98 | readme         | コミュニティ（Discord）                                             | 支援や告知のためのコミュニティチャンネルはありますか？                       | はい。https://discord.gg/hXCS9J6Qek からDiscordに参加してください。                                                                                                                                                          |
| 99 | readme         | コミュニティ（Discord）                                             | Discordは何のために使われていますか？                                        | サポートとプロジェクトの告知のためです。                                                                                                                                                                                     |